# <p style="text-align: center;"> <span style="color:yellowgreen"> MM camera </span></p>

In this notebook, we will cover information about the MM camera

## Observing the camera with rqt_image_view


The docker container from Lab 1 should already be in the MM.  
We just need to sync the new available code to the MM. 

1. In the HostOS terminal, go to the `lab-devel` directory using the `cd` command.  
Double check that you are in the `lab-devel` folder with the `pwd` command. 
It should return: `repo_folder/lab-devel`

2. To connect visuals to from the MM to the laptop monitor, we need to use the dts gui command.
This will allow to open rqt_image_view, or rqt_graph in the laptop computer based on the docker container in the MM.
To start the gui, tun the following command.


In [ ]:
dts start_gui_tools ee483mm<number>

Again this command allows us to open a new terminal tab that can connect visuals to the laptop.   
The usual dts devel run attach cannot connect to the notebook monitor while start_gui_tools can.  
Your terminal will look something similar to this:


<br><img src="../assets/lab2-images/dts-gui.png" style="width: 75%; height: auto;"></br>

You can only run gui in one terminal, i.e., you cannot have multiple terminals with start_gui_tools.  
If you have a terminal running with start_gui_tools and try to open a second one, you will get an error saying that you already have a terminal open with gui.  

3. Once the MM shell appears, run `rqt_image_view`  
The camera topic is called `ee483mm<number>/camera_node/image/compressed`  
When you select this topic in the rqt_image_view, you should see the camera image 
Make sure to remove the camera cover  
**You do not need to place it back, specially after the callibration**

4. You can close the `rqt_image_view` when done observing. You can follow these steps to reopen. 

5. Keep the gui terminal open, i.e., do not exit the container terminal after running `dts start_gui_tools ee483mm<number>`  
We will use it again 

## Image topic 


The image in the MM is `Compressed` in order to reduce the computational burden in embedded systems.  

1. Let's find out about this type. In the gui terminal (the one after running `dts start_gui_tools ee483mm<number>`), 

In [ ]:
rostopic info /ee483mm<number>/camera_node/image/compressed

You should see an image similar to the following:

<br><img src="../assets/lab2-images/compressed-image.png" style="width: 75%; height: auto;"></br>

**One difference in your node compared to Exs 4 and 5 are the convertion used in cv_bridge.**  
You will need to use a slightly different function in your OpenCV bridge: `self.bridge.compressed_imgmsg_to_cv2`  
There is no need to publish compressed images though.

Note that you can run debuging commands such as rostopic, rosnode, etc in this terminal.

2. You can close the gui terminal using the command `exit`.

## Image flip

Similar to Ex4, let's first create a simple node to flip the image coming from the MM.

### First we need to create your lab2 package
(DOESN'T WORK IF NOT ADM - COPY PACKAGE IS BETTER - OR CREATE PACKAGE BEFORE SENDING TO THEM)

If we create the package directly in the MM, it will not sync back to the laptop.  
For this reason, we will create the package in the Devel container in the laptop.  
Next, we will sync to the Devel container in the MM 


1. In the HostOS terminal, go to the `lab-devel` directory using the `cd` command.  
Double check that you are in the `lab-devel` folder with the `pwd` command. 
It should return: `repo_folder/lab-devel`

2. Building and running the image in the laptop first  
Similar to the exercise Devel container 


In [ ]:
dts devel build --pull
dts devel run -s -X -M --cmd bash -f 

3. Once the devel container shell appears, let's create a new package.
First go to the packages folder.

In [ ]:
cd packages

4. Create a new package with catkin_create_pkg

In [ ]:
catkin_create_pkg lab2 sensor_msgs cv_bridge rospy duckietown_msgs std_msgs

You should be able to see the folder `lab2` inside the `lab-devel/packages`  
Your `lab1`package is also there from Lab 1 

5. Exiting the Devel container in your laptop with command `exit`

### Creating the img_flip node

1. Create a node inside the src folder in the lab2 package. 
In VS Code in the Machine, right click in the src folder `New File...`: Name img_flip.py

2. Make it executable: `chmod +x packages/lab2/src/img_flip.py` command (assuming your terminal is inside the lab-devel folder). If it is not, you need to give the path, e.g., `chmod +x lab2/src/img_flip.py`, when the terminal is inside lab2 folder.

3. Adding the flip code to the `img_flip.py` node.

In [ ]:
#!/usr/bin/env python3

import sys
import rospy
import cv2
from sensor_msgs.msg import Image
from cv_bridge import CvBridge

class ImageFlipper:
    def __init__(self):
        # Instatiate the converter class once by using a class member
        self.bridge = CvBridge()
        rospy.Subscriber("image", Image, self.flipper_cb)
        self.pub = rospy.Publisher("mirrored", Image, queue_size=10)
    
    def flipper_cb(self, msg):
        # convert to a ROS image using the bridge
        cv_img = self.bridge.imgmsg_to_cv2(msg, "bgr8")
        
        # flip along the horizontal axis using an OpenCV function
        cv_flipped = cv2.flip(cv_img, 1)
        
        # convert new image to ROS to send
        ros_flipped = self.bridge.cv2_to_imgmsg(cv_flipped, "bgr8")
        
        # publish flipped image
        self.pub.publish(ros_flipped)
        

if __name__=="__main__":
    # initialize our node and create a publisher as normal
    rospy.init_node("image_flipper", anonymous=True)
    img_flip = ImageFlipper()
    rospy.spin()


This code is the same as in flip_image in the package img_proc_aux from Ex4  
**IT WILL NOT WORK DIRECTLY IN THE MM**

Changes you need to do

- Subscribe to the correct camera topic name
- Import CompressedImage message type
- Import os (to discover the name of the MM as you did in Lab1)
- Create Subscriber for CompressedImage: `rospy.Subscriber("mm_camera_topic_name", CompressedImage, self.flipper_cb, queue_size=1, buff_size=10000000) #BUFF SIZE 10MB`   
Although the Jetson Nano is a relatively powerful embedded system, your image processing code will probably take a relatively long time to run on the robot. 
The additional arguments in the subscriber ensures that your ROS is not processing queued images (old images)  
- Change the conversion method used to CompressedImage instead of Image, i.e., function `self.bridge.compressed_imgmsg_to_cv2(compressed_image)`  
Note that in the compressed image conversion, we do not add a encoding
- You can publish Image instead of CompressedImage in your publisher

4. Sync'ing the image in the MM (You need to be in the lab-devel folder)

In [ ]:
dts devel run -H ee483mm<number>.local -s -M --cmd bash -f 

5. Building and source the catkin workspace

In [ ]:
catkin_build
source /code/catkin_ws/devel/setup.bash

6. Running the node `rosrun lab2 img_flip.py`

7. Follow 'Observing the camera' to open rqt_image_view to see the flipped image  
You can see the camera_node/image/compressed and the mirrored image as below

<br><img src="../assets/lab2-images/camera_image.png" style="width: 35%; height: auto;"> <img src="../assets/lab2-images/camera_mirrored.png" style="width: 35%; height: auto;"> </br>